In [1]:
%load_ext autoreload
%autoreload 2
from __future__ import division, print_function, absolute_import
import pickle
import os
from ANEMO import read_edf
from ANEMO import ANEMO
import numpy as np
from lmfit import  Model, Parameters

In [2]:
sujet = ['AM','BMC','CS','DC','FM','IP','LB','OP','RS','SR','TN','YK']
time = ['2017-10-23_100057','2017-09-26_095637',
        '2017-10-03_134421','2017-09-27_161040',
        '2017-10-03_143803','2017-09-28_115250',
        '2017-09-20_151043','2017-10-26_121823',
        '2017-11-08_094717','2017-11-16_153313',
        '2017-11-08_150410','2017-11-17_172706']

In [3]:
equation='fct_velocity'
time_sup = 280
opt_base = {'stop_search_misac': None, 'time_sup': 280,
            'param_fit': None, 'inde_vars': None,
            'step_fit':2, 'do_whitening': False,
            'before_sacc': 5, 'after_sacc': 15}    

In [ ]:
x = 5
timeStr = time[x]
observer = sujet[x]
mode = 'enregistrement'

print(observer,timeStr)

file = os.path.join('data', mode + '_' + observer + '_' + timeStr + '.pkl')
with open(file, 'rb') as fichier :
    param_exp = pickle.load(fichier, encoding='latin1')

resultats = os.path.join('data', mode + '_' + observer + '_' + timeStr + '.asc')
data = read_edf(resultats, 'TRIALID')

A = ANEMO(param_exp)
Fit = ANEMO.Fit(param_exp)

IP 2017-09-28_115250


In [ ]:
N_trials = param_exp['N_trials']
N_blocks = param_exp['N_blocks']

goodness_of_fit = {}

for s in ['nfev', 'residual', 'chisqr', 'redchi', 'aic', 'bic'] :
    goodness_of_fit[s] = []

for block in range(N_blocks) :
    
    for s in ['nfev', 'residual', 'chisqr', 'redchi', 'aic', 'bic'] :
        goodness_of_fit[s].append([])
    
    for trial in range(N_trials) :

        print(block, trial)

        trial_data = trial + N_trials*block
        arg = A.arg(data[trial_data], trial=trial, block=block)
        opt = opt_base.copy()
        opt.update(arg)

        velocity_NAN = A.velocity_NAN(**opt)[0]

        #-------------------------------------------------
        # FIT
        #-------------------------------------------------
        if equation=='fct_velocity' :
            data_trial = velocity_NAN
        else :
            data_trial = A.data_deg(data=arg.data_x, **opt)

        old_latence, old_max, old_anti = A.classical_method.Full(velocity_NAN, arg.TargetOn-arg.t_0)
        f = Fit.Fit_trial(data_trial, equation=equation, value_latence=old_latence, value_max=old_max, value_anti=old_anti, **opt)
        #-------------------------------------------------
        goodness_of_fit['nfev'][block].append(f.nfev)
        goodness_of_fit['residual'][block].append(f.residual)
        goodness_of_fit['chisqr'][block].append(f.chisqr)
        goodness_of_fit['redchi'][block].append(f.redchi)
        goodness_of_fit['aic'][block].append(f.aic)
        goodness_of_fit['bic'][block].append(f.bic)

0 0
0 1
0 2
0 3
0 4
0 5
0 6
0 7
0 8
0 9
0 10
0 11
0 12
0 13
0 14
0 15
0 16
0 17
0 18
0 19
0 20
0 21
0 22
0 23
0 24
0 25
0 26
0 27
0 28
0 29
0 30
0 31
0 32
0 33
0 34
0 35
0 36
0 37
0 38
0 39
0 40
0 41
0 42
0 43
0 44
0 45
0 46
0 47
0 48
0 49
0 50
0 51
0 52
0 53
0 54
0 55
0 56
0 57
0 58
0 59
0 60
0 61
0 62
0 63
0 64
0 65
0 66
0 67
0 68
0 69
0 70
0 71
0 72
0 73
0 74
0 75
0 76
0 77
0 78
0 79
0 80
0 81
0 82
0 83
0 84
0 85
0 86
0 87
0 88
0 89
0 90
0 91
0 92
0 93
0 94
0 95
0 96
0 97
0 98
0 99
0 100
0 101
0 102
0 103
0 104
0 105
0 106
0 107
0 108
0 109
0 110
0 111
0 112
0 113
0 114
0 115
0 116
0 117
0 118
0 119
0 120
0 121
0 122
0 123
0 124
0 125
0 126
0 127
0 128
0 129
0 130
0 131
0 132
0 133
0 134
0 135
0 136
0 137
0 138
0 139
0 140
0 141
0 142
0 143
0 144
0 145
0 146
0 147
0 148
0 149
0 150
0 151
0 152
0 153
0 154
0 155
0 156
0 157
0 158
0 159
0 160
0 161
0 162
0 163
0 164
0 165
0 166
0 167
0 168
0 169
0 170
0 171
0 172
0 173
0 174
0 175
0 176
0 177
0 178
0 179
0 180
0 181
0 182
0 183
0 184


In [ ]:
file = os.path.join('parametre', 'goodness_of_fit_%s.pkl'%(observer))
with open(file, 'wb') as fichier:
    f = pickle.Pickler(fichier)
    f.dump(goodness_of_fit)

In [ ]:
goodness_of_fit